# Addresses in Paris

<font size="4">Retrieving and visualising the addresses of artists that expose in Paris</font>

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import sparql_dataframe
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt
import folium
import VizKG.visualize as vkg

In [ ]:
endpoint = "http://localhost:3030/Basart/sparql"

<font size="4">For retrieving from Basart the addresses we filter, with a SPARQL query, all artists that have exposed Paris, and we retrieve their address (as given in the catalogues) </font>

#### SPARQL

In [ ]:
q ="""

PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX crmdig: <http://www.ics.forth.gr/isl/CRMdig/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX aat: <http://vocab.getty.edu/aat/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>

# Addresses in Paris

SELECT DISTINCT ?personLabel ?address ?address_label ?coordinates WHERE  {
VALUES ?paris {<https://visualcontagions.unige.ch/resources/city/33661>}

    ?person crm:P14i_performed ?pursuit ;
        rdfs:label ?personLabel .
    ?pursuit a frbroo:F51_Pursuit ;
        crm:P7_took_place_at ?address .
    ?address a crm:E53_Place;
        rdfs:label ?address_label ;
        crm:P168_place_is_defined_by ?coordinates ;
        crm:P89_falls_within ?paris .

  
    FILTER NOT EXISTS {
    VALUES ?trash {<https://visualcontagions.unige.ch/resources/address/5605> } .
        ?pursuit crm:P7_took_place_at ?trash .
    
    }
}



"""

In [ ]:
df = sparql_dataframe.get(endpoint, q)

<font size="4">Once done we quickly compute how many people share an address </font>

In [ ]:
df['counts'] = df.groupby(['address_label'])['personLabel'].transform('count')

In [ ]:
a, b = 1, 10
x, y = df.counts.min(), df.counts.max()
df['SIZE'] = (df.counts - x) / (y - x) * (b - a) + a

In [ ]:
df

<font size="4">And we plot this information on a map of Paris</font>

<br>

In [ ]:
df['geometry'] = df.coordinates.apply(wkt.loads)
df.drop('coordinates', axis=1, inplace=True)
gdf = gpd.GeoDataFrame(df, geometry='geometry')
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in gdf.geometry]
gdf['lon'] = gdf['geometry'].x
gdf['lat'] = gdf['geometry'].y
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in gdf.geometry]

In [ ]:
map = folium.Map(tiles="cartodbpositron", location=[48.8598854,2.3427464], zoom_start=13)
i = 0
for coordinates in geo_df_list:
    # Place the markers with the popup labels and data
    map.add_child(
        folium.CircleMarker(
            location=coordinates,
            radius=2,
            color="red", weight=1, fill_color="red", fill_opacity=0.7,
            popup=
                "Artist: " + str(gdf.personLabel[i]) + "<br>"
                + "Address: " + str(gdf.address_label[i]),
        )
    )
    i = i + 1

In [ ]:
map

<br>
<font size="4">We can also check which are the most used addresses by the artist in the city</font>

<br>

In [ ]:
map = folium.Map(tiles="cartodbpositron", location=[48.8598854,2.3427464], zoom_start=13)
i = 0
for coordinates in geo_df_list:
    # Place the markers with the popup labels and data
    map.add_child(
        folium.CircleMarker(
            location=coordinates,
            radius=float(gdf.iloc[i]['SIZE']),
            color="red", weight=1, fill_color="red", fill_opacity=0.7,
            popup=
                "Artist: " + str(gdf.personLabel[i]) + "<br>"
                + "Address: " + str(gdf.address_label[i]),
        )
    )
    i = i + 1

In [ ]:
map